<div style="text-align: right">INFO 6106 Neural Modeling, Lecture 11 Day 2</div>
<div style="text-align: right">Dino Konstantopoulos, 29 March 2023</div>

<br />
<left>
<img src="ipynb.images/rnn.avif" width=950 />
</left>

# About Tensorflow and Mac versions
If you're using version 2.1 or higher of tensorflow, you're going to have to modify some of the code in your neural network. Specifically, the line below:
```
preds = model.predict_classes(rowx, verbose=0)
```
needs to become:
```
preds = np.argmax(model.predict(rowx), axis=-1)
```

Also, if you're running on Mac/M1, you need to ***reinstall your Anaconda distribution from here***:
```
https://github.com/conda-forge/miniforge
```

Because M1 does not work with standard Anaconda! All you `keras` or `tensorflow` cells with kill your kernel!

Also, you need to downgrade from python 3.9 to python 3.8 (many libraries still remain unsupported on 3.9)
```
conda install python==3.8
```

For M1 users who wants to use GPU acceleration, please follow:

https://developer.apple.com/metal/tensorflow-plugin/


# Sequence to Sequence Networks

Much data is in the form of ***sequences***: From number sequences (time series), to image pixel sequences (pictures), picture frame sequences (video), audio sequences (music), text sequences (translations). And Chicago LES, too!

<br />
<left>
<img src="ipynb.images/sequence-models-2.png" width=600 />
</left>

Seq2seq models are an **encoder-decoder** architecture. The encoder processes the input sequence and encodes/compresses/summarizes the information into a **context vector** (I call it a *thought vector*) of a fixed length.

This representation is expected to be a good summary of the entire input sequence.

Your brain does the same thing! When you hear music, watch video, or eat food one spoonful at a time, that represents **information**, which your brain encodes with electric and chemical signals in your neurons. Just like your brain has internal data representations, so do RNNs!

So, the encoder is the part of the engine that encodes the information into a **context vector**. The decoder is then initialized with this context vector, to recover the initial input. The recovering operation is there to **interpolate** data. So that we can predict soemthing when the data is missing and the network was not trained with it. 

>**For example**, that is how cooks come up with new recipes: They interpolate between the taste of **sweet**, and the taste of **sour**, and imagine that **sweet and sour** would also be a good idea: The taste decoder tells the cook: Go for it!

Sequence to Sequence (abbreviated seq2seq or s2s) models are a special class of **Recurrent Neural Network** architectures typically used (but not restricted) to solve complex language-related problems like Machine Translation, Question Answering, creating Chat-bots, Text Summarization, etc.

For example, if $Y$ is a sentence in Chinese and $X$ is the same sentence in english, then the probable value for $Y$ is:

$$P(Y\;|\;X) = P(Y\;|\;x_1, x_2, \cdots, x_n)$$

where the $x_i$ are internal state representations of the english sentence $X$. It's a **lower-dimensional model** for $X$. This model is called the **encoder**.

And since the chinese sentence $Y$ is made out of a series of **hanzi** $y_1, y_2, y_3, \cdots, y_m$, then we write:

$$P(Y\;|\;x_1, x_2, \cdots, x_n) = P(y_i|y_1, y_2, \cdots y_{i-1}\;;\;x_1, x_2, \cdots, x_n)$$

In other words, the probablity of translated $y_i$ is conditioned upon previously translated words $y_0$ to $y_{i-1}$ and hidden state output $x_i$ for the encoder. The translated words are output **one by one** by the **decoder**.

Recurrent Neural Networks (or more precisely LSTM/GRU) have been found to be very effective in modeling this formula.


# Recurrent Neural Networks (RNN)

<br />
<left>
<img src="ipynb.images/rnn-vs-fnn.png" width=500 />
</left>

RNNs have a simple API: They accept an input vector $x$ and give you an output vector $y$. However, this output vector’s contents are influenced *not only by the input you just fed in*, but also ***on the entire history of inputs you’ve fed in in the past***. Think of RNNs as a **loop** over the dense ANNs we learned last week, where at each iteration of the loop, we use the output of the previous iteration ***as well as a new input vector*** as input to the next iteration.

<br />
<left>
<img src="ipynb.images/rnn.png" width=500 />
</left>

Recurrent neural networks are networks ***with loops in them***, allowing state information to persist. The loop allows information to be passed from one step of the network to the next. A recurrent neural network can be thought of as multiple copies of the same network, each passing a message to a successor.

<br />
<left>
<img src="ipynb.images/RNN-unrolled.png" width=600 />
</left>

It's a bit like this:

<br />
<left>
<img src="ipynb.images/tin-can-phone.jpg" width=400 />
</left>

Here is an example character-based RNN by Andrej Karpathy. We just replace integer digits from last week's notebook to character digits. And now all we're trying to do is to *predict* the next character of an english sentence.

<br />
<left>
<img src="ipynb.images/hello-predictor.jpeg" width=500 />
</left>

Sometimes, we only need to look at ***recent*** information to perform the present task. In such cases, where the gap between the relevant information and the place that it’s needed is **small**, RNNs can learn to use past information pretty successfully.

<br />
<left>
<img src="ipynb.images/simpleRNN.png" width=600 />
</left>

But there are also cases where we need more context. It’s possible for the gap between the relevant information and the point where it is needed to become **very large**. As that gap grows, RNNs become unable to learn to connect the information.

# A 1-hidden-layer RNN

Written as a python class, the RNN’s API consists of a single step function, which is the forward pass of the RNN. 

Assuming an input layer of neurons $x$, a hidden layer of neurons $h$, and an output layer of neurons $y$, this RNN’s parameters are the three matrices $W_{hh}$, $W_{xh}$, and $W_{hy}$. 

The hidden state `self.h` is initialized with the zero vector.  

This RNN’s parameters are the three matrices $W_{hh}$, $W_{xh}$, and $W_{hy}$. 

The matrix $W_{xh}$ transforms a new data input at every time step. The matrix $W_{hy}$ gives us the output in the form of logits (probabilities per token). These are the matrices we're used to with dense networks.

The matrix $W_{hh}$ is the new matrix that is assocaited with networks that are *recurrent*. It transforms the previous timestep to the next time step. 

The `np.tanh` function implements a non-linearity that squashes the activations to the range [-1, 1].

There will be *two* terms inside of the `tanh`: One is based on the current input and one is based on the previous hidden state. The two intermediates interact with addition, and then get squashed by the tanh into the new state vector.

$$h_t = \tanh ( W_{hh} h_{t-1} + W_{xh} x_t )$$ 

We initialize the matrices of the RNN with random numbers and backpropagation training consists in finding the matrices that give rise to desirable behavior, as measured with some loss function that expresses your preference to what kinds of outputs $y$ you’d like to see in response to your input sequences $x$.

Here's some simple code for a 1 hidden layer RNN.

# Feedforward code for a 1-hidden-layer RNN
We implement a 3-3 RNN: One hidden layer of 3 neurons, and one output layer with 3 neurons.

Since we now have an RNN, that RNN has an internal state that changes with each timestep. We call that state `h`. Since our hidden layer has 3 neurons, our state is a 3D vector.

Each observation consists of two columns (i.e. two *steps*).

We input the observation $[1, 0]$ and we look for the (untrained) output.

In [1]:
import numpy as np

class RNN:
    def __init__(self):
        self.W_xh = np.random.rand(2,3)
        self.W_hh = np.random.rand(3,3)
        self.W_hy = np.random.rand(3,3)
        self.h = np.zeros(3)

    def step(self, x):
        # update the hidden state
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh.T, x))

        # compute the output vector
        y = np.dot(self.W_hy, self.h)

        return y

rnn = RNN()
x = np.array([1,0])
y = rnn.step(x) # x is an input vector, y is the RNN's output vector
y

array([0.11914044, 0.1626241 , 0.10370503])

We initialize the matrices of the RNN with random numbers and **backpropagation training** consists in finding the matrices that give rise to desirable behavior, as measured with some loss function that expresses your preference to what kinds of outputs $y$ you’d like to see in response to your input sequences $x$. That is **supervised ML**.

# (An idea about) the general Math of RNNs
A more general RNN may actually work with sequences of 5 steps:

In this case, we don't have one simple loss, but in fact we have 5 losses (pictured as `E`s for `E`rrors) here below:

<br />
<left>
<img src="ipynb.images/5-step-rnn.png" width=500 />
</left>

We just sum up the errors and compute gradients using the sum of the errors:

$$\frac{\partial E}{\partial W^3} = \sum_{i=0}^4 \frac{\partial E_i}{\partial W^3}$$

Let's write down the backpropagation equations for the last layer with weights $W^3$, and let's just study the loss $E_3$:

$$\frac{\partial E_3}{\partial W^3} = \frac{\partial E_3}{\partial \hat{y}_3} * \frac{\partial \hat{y}_3}{\partial W^3} $$
 
If $z_3$ is the output of the network before activation $\sigma$, then $y_3 = \sigma(z_3)$ and we write: 

$$\frac{\partial E_3}{\partial W^3} = \frac{\partial E_3}{\partial \hat{y}_3} * \frac{\partial \hat{y}_3}{\partial z_3} * \frac{\partial z_3}{\partial W^3} $$

If $z_3 = W^3 s_3$, then:

$$\frac{\partial E_3}{\partial W^3} = 2(y_3 - \hat{y}_3) * \sigma'(z_3) \otimes s_3 $$

Ok, so the output layer was pretty straightforward. Now, let's write down the backpropagation for the hidden layer with weights $W^2$:

$$\frac{\partial E_3}{\partial W^2} = \frac{\partial E_3}{\partial \hat{y}_3} * \frac{\partial \hat{y}_3}{\partial s_3} * \frac{\partial s_3}{\partial W^2} $$

But now $s_3 = \sigma(W^1 x + W^2s_2)$. So $s_3$ depends on $s_2$. Likewise, $s^2$ depends on $s_1$, and $s_1$ depends on $s_0$. Because $W^2$ is used in every step up to the output we care about, we need to backpropagate gradients from $t=3$ through the network all the way to $t=0$:

<br />
<left>
<img src="ipynb.images/5-step-rnn-bptt.png" width=500 />
</left>

In a traditional dense ANN we don’t share parameters across layers, so we don’t need to worry about inteacting weights across layers. But because of the state update equation, now we need to. And we write:

$$\frac{\partial E_3}{\partial W^2} = \sum_{k=0}^3 \frac{\partial E_3}{\partial \hat{y}_3} * \frac{\partial \hat{y}_3}{\partial s_3} * \frac{\partial s_3}{\partial s_k} * \frac{\partial s_k}{\partial W^2} $$

where, for example:

$$\frac{\partial s_3}{\partial s_1} = \frac{\partial s_3}{\partial s_2} * \frac{\partial s_2}{\partial s_1}$$

Because we are taking the derivative of a vector function $s_i$ with respect to another vetor function $s_k$, the result is a matrix (called the Jacobian matrix) whose elements are all pointwise derivatives:

$$\frac{\partial s_3}{\partial W^2} = \left( \prod_{j=k+1}^3 \frac{\partial s_j}{\partial s_{j-1}} \right) \frac{\partial s_k}{\partial W^2}$$

and so on for the other $s_k$s.

This is why standard RNNs are hard to train: Sequences (i.e. sentences of many words) can be quite long, perhaps 20 words or more, and thus you need to back-propagate through 20 layers (n practice many people truncate the backpropagation to a few steps)! How would you like to write the math for backpropagation through 20 layers?!

As a corollary, RNNs have difficulties learning long-range dependencies – interactions between words that are several steps apart. That’s problematic because the meaning of an English sentence is often determined by words that aren’t very close: “*The woman who liked the man took off the hat and smiled*”. The sentence is really about a woman, not about a man. But it’s likely that a plain RNN would think the man took off the hat and smiled, instead. 

That is because sigmoid functions have derivatives of 0 near 0 and 1 potentials (they approach a  flat line, the opposite behavior of their primitives). When this happens we say the corresponding neurons are saturated. They have a zero gradient and drive other gradients in previous layers towards 0. Thus, with small values in the matrix and multiple matrix multiplications later, the gradient values are shrinking exponentially fast, eventually vanishing completely after a few time steps. 

Gradient contributions from “far away” steps become zero, and the state at those steps doesn’t contribute to what you are learning: You end up not learning long-range dependencies. Vanishing gradients aren’t exclusive to RNNs. They also happen in deep Feedforward Neural Networks. It’s just that RNNs tend to be very deep (as deep as the sentence length in our case), which makes the problem common.

And then, to top this off, you need to worry about the additional complexity of multiple hidden layers, too!

# How ChatGPT would work if it had only 1 hidden layer

ChatGPT is based on a *Transformer* architecture, which we'll introduce in a few weeks. But we can program a chatbot with an RNN, and it's very instructive.

## Language Modeling

Our goal is to build a [Language Model](https://en.wikipedia.org/wiki/Language_model) using a Recurrent Neural Network. Here's what that means. Let's say we have sentence  of $m$ words. Language Model allows us to predict the probability of observing the sentence (in a given dataset) as:

$
\begin{aligned}
P(w_1,...,w_m) = \prod_{i=1}^{m}P(w_i \mid w_1,..., w_{i-1}) 
\end{aligned}
$

In words, the probability of a sentence is the product of probabilities of each word given the words that came before it. 

Why is that useful? Why would we want to assign a probability to observing a sentence?

A Machine Translation system typically generates multiple candidates for an input sentence. You could use a language model to pick the most probable sentence. Similar scoring happens in speech recognition systems.

But solving the Language Modeling problem also has a cool side effect. Because we can predict the probability of a word given the preceding words, we are able to generate new text. It's a *generative model*. 

Given an existing sequence of words we sample a next word from the predicted probabilities, and repeat the process until we have a full sentence. That is how chatGPT works. It does not form an opinion, *first*, as humans do. It just starts outputting words, and, *word by word*, generates something *that makes sense*!

Note that in the above equation the probability of each word is conditioned on **all** previous words. In practice, many models have a hard time representing such long-term dependencies due to computational or memory constraints. They are typically limited to looking at only a few of the previous words. 

In [2]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Download NLTK model data (you need to do this once)
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\chat80.zip.
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\conll2000.zip.
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]    |   U

True

## Training Data and NLP Preprocessing
Let's read and integer-tokenize some text.

To train our language model we need text to learn from. 

### 1. Tokenize Text

We have raw text, but we want to make predictions on a per-word basis. This means we must *tokenize* our comments into sentences, and sentences into words. We could just split each of the comments by spaces, but that wouldn't handle punctuation properly. The sentence "He left!" should be 3 tokens: "He", "left", "!". We'll use [NLTK's](http://www.nltk.org/) `word_tokenize` and `sent_tokenize` methods, which do most of the hard work for us.

### 2. Remove infrequent words

Most words in our text will only appear one or two times. It's a good idea to remove these infrequent words. Having a huge vocabulary will make our model slow to train (we'll talk about why that is later), and because we don't have a lot of contextual examples for such words we wouldn't be able to learn how to use them correctly anyway. That's quite similar to how humans learn. To really understand how to appropriately use a word you need to have seen it in different contexts.

In our code we limit our vocabulary to the `vocabulary_size` most common words (which I set to 8000, but feel free to change it). We replace all words not included in our vocabulary by `UNKNOWN_TOKEN`. For example, if we don't include the word "nonlinearities" in our vocabulary, the sentence "nonlineraties are important in neural networks" becomes "UNKNOWN_TOKEN are important in Neural Networks". The word `UNKNOWN_TOKEN` will become part of our vocabulary and we will predict it just like any other word. When we generate new text we can replace `UNKNOWN_TOKEN` again, for example by taking a randomly sampled word not in our vocabulary, or we could just generate sentences until we get one that doesn't contain an unknown token.

### 3. Prepend special start and end tokens

We also want to learn which words tend start and end a sentence. To do this we prepend a special `SENTENCE_START` token, and append a special `SENTENCE_END` token to each sentence. This allows us to ask: Given that the first token is `SENTENCE_START`, what is the likely next word (the actual first word of the sentence)?


### 4. Build training data matrices

The input to our Recurrent Neural Networks are vectors, not strings. So we create a mapping between words and indices, `index_to_word`, and `word_to_index`. For example,  the word "friendly" may be at index 2001. A training example $x$ may look like `[0, 179, 341, 416]`, where 0 corresponds to `SENTENCE_START`. The corresponding label $y$ would be `[179, 341, 416, 1]`. Remember that our goal is to predict the next word, so y is just the x vector shifted by one position with the last element being the `SENTENCE_END` token. In other words, the correct prediction for word `179` above would be `341`, the actual next word.

You can either play with Web data:

In [99]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print( "Reading CSV file...")
with open(r'data\reddit-comments-2015-08.csv', 'r', encoding="utf-8") as f:
    reader = csv.reader(f, skipinitialspace=True)
    #reader.next()
    
    # Split full comments into sentences
    #sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in next(reader)])
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    
print(  "Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print(  "Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print(  "\nExample sentence: '%s'" % sentences[0])
print(  "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

Reading CSV file...
Parsed 79062 sentences.
Found 63030 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'appointments' and appeared 10 times.

Example sentence: 'SENTENCE_START body SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'body', 'SENTENCE_END']'


Or with Shakeaspeare's sonnets:

In [84]:
import re
from nltk import tokenize

#alphabets= "([A-Za-z])"
#prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
#suffixes = "(Inc|Ltd|Jr|Sr|Co)"
#starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
#acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
#websites = "[.](com|net|org|io|gov|edu|me)"
#digits = "([0-9])"

#vocabulary_size = 8000
vocabulary_size = 3000

unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

def clean_roman_numerals(text):
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    return re.sub(pattern, '&', text)

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print( "Reading txt file...")
with open(r'data\shakespeare-sonnets.txt', 'r') as f:
    text = f.read()
    
    text = text.replace(",",".")
    text = text.replace(":",".")
    text = text.replace(";",".")
    text = text.replace("?",".")
    text = text.replace("!",".")
    
    text = clean_roman_numerals(text)
    
    #text = re.sub(prefixes,"\\1<prd>",text)
    #text = re.sub(websites,"<prd>\\1",text)
    #text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    #if "..." in text: text = text.replace("...","<prd><prd><prd>")
    #if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    #text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    #text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    #text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    #text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    #text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    #if "”" in text: text = text.replace(".”","”.")
    #if "\"" in text: text = text.replace(".\"","\".")
    #if "!" in text: text = text.replace("!\"","\"!")
    #if "?" in text: text = text.replace("?\"","\"?")
    #text = text.replace(".",".<stop>")
    #text = text.replace("?","?<stop>")
    #text = text.replace("!","!<stop>")
    #text = text.replace("<prd>",".")
    #sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    #sentences = [s.strip() for s in sentences]
    
    sentences = tokenize.sent_tokenize(text)
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    
print(  "Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print(  "Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print(  "\nExample sentence: '%s'" % sentences[0])
print(  "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

Reading txt file...
Parsed 2766 sentences.
Found 3400 unique words tokens.
Using vocabulary size 3000.
The least frequent word in our vocabulary is 'miss' and appeared 1 times.

Example sentence: 'SENTENCE_START &

From fairest creatures we desire increase. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', '&', 'From', 'fairest', 'creatures', 'we', 'desire', 'increase', '.', 'SENTENCE_END']'


In [139]:
len(tokenized_sentences)

79062

In [143]:
tokenized_sentences[5]

['SENTENCE_START',
 'i',
 'put',
 'in',
 'the',
 'rules',
 'at',
 'a',
 'ranking',
 'site',
 'and',
 'noticed',
 'that',
 'top',
 'qbs',
 'had',
 '300',
 'points',
 'more',
 'than',
 'the',
 'top',
 'UNKNOWN_TOKEN',
 '.',
 'SENTENCE_END']

In [86]:
sentences[1:5]

['SENTENCE_START That thereby beauty’s rose might never die. SENTENCE_END',
 'SENTENCE_START But as the riper should by time decease. SENTENCE_END',
 'SENTENCE_START His tender heir might bear his memory. SENTENCE_END',
 'SENTENCE_START But thou. SENTENCE_END']

Let's create the training data:

In [196]:
X_train = []
y_train = []
for idx,sent in enumerate(tokenized_sentences[:-1]):
    X_train.append([word_to_index[w] for w in sent])
    y_train.append([word_to_index[w] for w in tokenized_sentences[idx+1]][:len([word_to_index[w] for w in sent])])

In [231]:
X_train

array([list([1, 675, 2999, 2999, 34, 675, 33, 598, 174, 2105, 2999, 42, 40, 8, 3]),
       list([1, 2999, 598, 159, 402, 2999, 1787, 3]),
       list([1, 422, 88, 0, 932, 0, 198, 234, 4, 2967, 892, 3]), ...,
       list([1, 2999, 25, 738, 24, 2999, 2999, 21, 2999, 2999, 2999, 12, 460, 6, 22, 696, 2999, 5, 66, 1144, 106, 2999, 2999, 3]),
       list([1, 2254, 0, 2999, 2999, 2999, 232, 606, 2999, 319, 6, 62, 12, 460, 6, 480, 21, 860, 1314, 186, 2999, 2679, 2999, 2999, 2999, 2999, 2870, 4, 936, 2999, 2999, 0, 144, 537, 18, 9, 1370, 428, 2999, 23, 2999, 2999, 2999, 2999, 12, 20, 150, 15, 394, 2999, 668, 8, 14, 2999, 254, 6, 312, 254, 0, 30, 947, 248, 5, 15, 2999, 17, 5, 163, 18, 3]),
       list([1, 2999, 2999, 2999, 1043, 0, 1247, 921, 2999, 12, 94, 14, 299, 2999, 5, 196, 4, 2999, 2999, 0, 2999, 2999, 52, 5, 66, 720, 52, 13, 56, 6, 2999, 169, 102, 28, 50, 95, 2999, 2999, 2999, 2999, 70, 48, 85, 143, 5, 67, 308, 2330, 2999, 1043, 11, 2999, 2999, 481, 1926, 10, 2388, 51, 15, 25, 103, 11])],

In [233]:
np.array([[1,2,3],[1,2],[1]])

C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\967168167.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([[1,2,3],[1,2],[1]])


array([list([1, 2, 3]), list([1, 2]), list([1])], dtype=object)

In [237]:
pip show pandas

Name: pandas
Version: 1.4.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: c:\users\abhis\anaconda3\lib\site-packages
Requires: numpy, python-dateutil, pytz
Required-by: arviz, datashader, holoviews, hvplot, pandas-datareader, pymc3, seaborn, statsmodels, stockstats, xarray
Note: you may need to restart the kernel to use updated packages.


In [232]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [198]:
X_train.shape

(79061,)

In [199]:
y_train.shape

(79061,)

In [100]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\540999602.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\540999602.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])


In [140]:
X_train.shape

(79062,)

In [141]:
type(X_train)

numpy.ndarray

Let's print a training data example

In [200]:
x_example, y_example = X_train[1000], y_train[1000]
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print ("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START * SENTENCE_END
[0, 15, 1]

y:
SENTENCE_START * *
[0, 15, 15]


`word_dim` is the size of our vocabulary, and `hidden_dim` is the size of our hidden layer (we can pick it).

Let's get concrete and see what the RNN for our language model looks like. The input $x$ will be a sequence of words (just like the example printed above) and each $x_t$ is a single word. But there's one more thing: Because of how matrix multiplication works we can't simply use a word index (like 36) as an input. Instead, we represent each word as a *one-hot vector* of size `vocabulary_size`. 

For example, the word with index 36 would be the vector of all 0's and a 1 at position 36. So, each $x_t$ will become a vector, and $x$ will be a matrix, with each row representing a word. 

We'll perform this transformation in our Neural Network code instead of doing it in the pre-processing. The output of our network $o$ has a similar format. Each $o_t$ is a vector of `vocabulary_size` elements, and each element represents the probability of that word being the next word in the sentence.

Let's recap the equations for the RNN:

$
\begin{aligned}
s_t &= \tanh(Ux_t + Ws_{t-1}) \\
o_t &= \mathrm{softmax}(Vs_t)
\end{aligned}
$

Let's assume we pick a vocabulary size $C = 3000$ and a hidden layer size $H = 100$. You can think of the hidden layer size as the "memory" of our network. Making it bigger allows us to learn more complex patterns, but also results in additional computation. Then we have:

$
\begin{aligned}
x_t & \in \mathbb{R}^{3000} \\
o_t & \in \mathbb{R}^{3000} \\
s_t & \in \mathbb{R}^{100} \\
U & \in \mathbb{R}^{100 \times 3000} \\
W & \in \mathbb{R}^{100 \times 100} \\
V & \in \mathbb{R}^{3000 \times 100} \\
\end{aligned}
$

$U,V$ and $W$ are the weights of our network. Note that they are the transpose of the weights we have used so far in this version of the code (so the first dimension represents the number of neurons in the downstream layer, not the upstram one). They are the parameters we want to learn from data. Thus, we need to learn a total of $2HC + H^2$ parameters. 

In the case of $C=3000$ and $H=100$ that's $610,000$.  

The dimensions also tell us the bottleneck of our model. Note that because $x_t$ is a one-hot vector, multiplying it with $U$ is essentially the same as selecting a column of U, so we don't need to perform the full multiplication. 

Then, the biggest matrix multiplication in our network is $Vs_t$. That's why we want to keep our vocabulary size small if possible.

Initializing the parameters $U,V$ and $W$ can be tricky. 

We can't just initialize them to 0's because that would result in symmetric calculations in all our layers. We must initialize them randomly. 

Because proper initialization seems to have an impact on training results there has been lot of research in this area. It turns out that the best initialization depends on the activation function ($\tanh$ in our case) and one [recommended](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf) approach is to initialize the weights randomly in the interval from $\left[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}\right]$ where $n$ is the number of incoming connections from the previous layer. 

Mostly, as long as you initialize your parameters to small random values it typically works out fine.

In [201]:
class RNN:    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

Let's implement forward propagation:

In [202]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [203]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
        
    return [o, s]

RNN.forward_propagation = forward_propagation

We not only return the calculated outputs, but also the hidden states. We will use them later to calculate the gradients, and by returning them here we avoid duplicate computation. Each `o`  is a vector of probabilities representing the words in our vocabulary, but sometimes, for example when evaluating our model, all we want is the next word with the highest probability. We call this function `predict`:

In [204]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNN.predict = predict

Let's try and see an example output:

In [205]:
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in X_train[10]]), X_train[10]))

x:
SENTENCE_START a dishonest seller is n't going to run the check in the first place . SENTENCE_END
[0, 7, 7510, 3415, 13, 17, 126, 5, 327, 3, 329, 14, 3, 132, 269, 2, 1]


In [206]:
np.random.seed(17)
model = RNN(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print (o.shape, o)

(17, 8000) [[0.00012533 0.00012507 0.00012494 ... 0.00012532 0.00012468 0.00012491]
 [0.00012559 0.00012507 0.00012454 ... 0.00012468 0.00012554 0.0001247 ]
 [0.00012441 0.0001249  0.00012564 ... 0.00012454 0.00012557 0.00012583]
 ...
 [0.00012512 0.00012408 0.00012532 ... 0.00012474 0.00012453 0.00012506]
 [0.00012453 0.00012502 0.00012459 ... 0.0001257  0.00012512 0.00012486]
 [0.00012453 0.0001246  0.00012504 ... 0.00012449 0.00012595 0.00012435]]


For each word in the sentence (16 above), our model made 8000 predictions representing probabilities of the next word. 

That is because we picked a vocabulary size $C = 8000$, so our output layer has $8000$ neurons.

The following gives the indices of the highest probability predictions for each word:

In [157]:
for i in predictions:
    print(i)

6290
4828
2617
6258
5316
7736
2669
758
996
1182
6020
1158
1510
5704
5019
1852


In [158]:
len(index_to_word)

8000

In [207]:
predictions = model.predict(X_train[10])
print (predictions.shape, predictions)
print ("x:\n%s" % (" ".join([index_to_word[x] for x in predictions])))

(17,) [6290 4828 2617 6258 5316 7736 2669  758  996 1182 6020 1158 1510 5704
 5019 1852 5537]
x:
20my cycles updated reserve freak v. license multiple faster hair borderlands classes holding summons atheist absolute increasingly


Ok, all parameters are as of yet untrained.

## Calculating the Loss

To train our network we need a way to measure the errors it makes. We call this the loss function $L$, and our goal is find the parameters $U,V$ and $W$ that minimize the loss function for our training data. 

A common choice for the loss function is the [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression). 

If we have $N$ training examples (words in our text) and $C$ classes (the size of our vocabulary) then the loss with respect to our predictions $o$ and the true labels $y$ is given by:

$
\begin{aligned}
L(y,o) = - \frac{1}{N} \sum_{n \in N} y_{n} \log o_{n}
\end{aligned}
$

The formula looks a bit complicated, but all it really does is sum over our training examples and add to the loss based on how off our prediction are. The further away $y$ (the correct words) and $o$ (our predictions), the greater the loss will be. We implement the function `calculate_loss`:

In [208]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
#         print('o ka shape ',o.ndim)
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
#         correct_word_predictions = o[np.arange(len(y[i]))]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNN.calculate_total_loss = calculate_total_loss
RNN.calculate_loss = calculate_loss

What should the loss be should be for random predictions? 

That will give us a baseline and make sure our implementation is correct. 

We have $C$ words in our vocabulary, so each word should be (on average) predicted with probability $1/C$, which would yield a loss of $L = -\frac{1}{N} N \log\frac{1}{C} = \log C$:

In [195]:
type(np.arange(len(y_train[0])),y_train[0])

TypeError: type() takes 1 or 3 arguments

In [176]:
y_train[8]

[0,
 60,
 1120,
 3415,
 13,
 126,
 5,
 29,
 719,
 3,
 315,
 5,
 71,
 68,
 33,
 112,
 33,
 76,
 7,
 4747,
 24,
 3,
 1011,
 329,
 2,
 1]

In [209]:
# Limit to 1000 examples to save time
print ("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print ("Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000]))

Expected Loss for random predictions: 8.987197


C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\1920175922.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  N = np.sum((len(y_i) for y_i in y))


Actual loss: 8.987136


## Training with Backpropagation Through Time

We iterate over all our training examples and during each iteration we nudge the parameters into a direction that reduces the error. 

These directions are given by the gradients on the loss: $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$. 

We also need a *learning rate*, which defines how big of a step we want to make in each iteration. 

Because the layer weight parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps!

We take as input a training example $(x,y)$ and return the gradients $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$.

In [210]:
def bptt(self, x, y):
    T = len(y)
    
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
            
    return [dLdU, dLdV, dLdW]

RNN.bptt = bptt

## Gradient Checking

Whenever you implement backpropagation it is good idea to also implement *gradient checking*, which is a way of verifying that your implementation is correct. The idea behind gradient checking is that derivative of a parameter is equal to the slope at the point, which we can approximate by slightly changing the parameter and then dividing by the change:

$
\begin{aligned}
\frac{\partial L}{\partial \theta} \approx \lim_{h \to 0} \frac{J(\theta + h) - J(\theta -h)}{2h}
\end{aligned}
$

We then compare the gradient we calculated using backpropagation to the gradient we estimated with the method above. If there's no large difference we are good. The approximation needs to calculate the total loss for *every* parameter, so that gradient checking is very expensive (remember, we had more than a million parameters in the example above). So it's a good idea to perform it on a model with a smaller vocabulary.

In [211]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
               
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
               
            # Save the original value so we can reset it later
            original_value = parameter[ix]
               
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
               
            # Reset parameter to original value
            parameter[ix] = original_value
               
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
               
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient) / (
                                np.abs(backprop_gradient) + np.abs(estimated_gradient))
            
               # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print( "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print( "+h Loss: %f" % gradplus)
                print( "-h Loss: %f" % gradminus)
                print( "Estimated_gradient: %f" % estimated_gradient)
                print( "Backpropagation gradient: %f" % backprop_gradient)
                print( "Relative Error: %f" % relative_error)
                return 
            it.iternext()
               
        print( "Gradient check for parameter %s passed." % (pname))

RNN.gradient_check = gradient_check

To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.

In [212]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNN(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\2693848989.py:38: RuntimeWarning: invalid value encountered in double_scalars
  relative_error = np.abs(backprop_gradient - estimated_gradient) / (


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


## Gradient Descent Implementation

A function `sdg_step` calculates the gradients and performs the updates for one batch. 

Then, an outer loop that iterates through the training set and adjusts the learning rate.

In [213]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNN.sgd_step = numpy_sdg_step

In [214]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs

def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    
    for epoch in range(nepoch):
        
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print ("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
            
        # For each training example...
        for i in range(len(y_train)):
            
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

Let's get a sense of how long it would take to train our network:

In [215]:
np.random.seed(17)
model = RNN(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

166 ms ± 21.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


One step of gradient descent takes approximately 33 milliseconds (120 miliseconds for Web data) on my laptop.

We have about 2,766 examples (80,000 examples if you use Web data) in our training data, so one epoch (iteration over the whole data set) would take tends of minutes (several hours with Web data). 

Multiple epochs would take many hours (days, or even weeks for Wen data!) And we're still working with a small dataset compared to what's being used by the OpenAIs, Googles and Facebooks!

Researchers have identified many ways to make models less computationally expensive, for example by using a hierarchical softmax or adding projection layers to avoid large matrix multiplications (see also [here](http://arxiv.org/pdf/1301.3781.pdf) or [here](http://www.fit.vutbr.cz/research/groups/speech/publi/2011/mikolov_icassp2011_5528.pdf)). 

Let's try to run SGD with a small dataset and check if the loss actually decreases:

In [216]:
vocabulary_size

8000

In [217]:
np.random.seed(17)

# Train on a small subset of the data to see what happens
model = RNN(vocabulary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\1920175922.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  N = np.sum((len(y_i) for y_i in y))


2023-04-02 15:33:52: Loss after num_examples_seen=0 epoch=0: 8.986899
2023-04-02 15:34:08: Loss after num_examples_seen=100 epoch=1: 8.966209
2023-04-02 15:34:24: Loss after num_examples_seen=200 epoch=2: 8.925260
2023-04-02 15:34:39: Loss after num_examples_seen=300 epoch=3: 8.822493
2023-04-02 15:34:55: Loss after num_examples_seen=400 epoch=4: 8.544621
2023-04-02 15:35:11: Loss after num_examples_seen=500 epoch=5: 6.406279
2023-04-02 15:35:27: Loss after num_examples_seen=600 epoch=6: 5.854575
2023-04-02 15:35:43: Loss after num_examples_seen=700 epoch=7: 5.588366
2023-04-02 15:35:59: Loss after num_examples_seen=800 epoch=8: 5.428171
2023-04-02 15:36:15: Loss after num_examples_seen=900 epoch=9: 5.318619


Yay! Our loss is monotonically decreasing :-)

## Generating Text

Now that we have a model, albeit not a very trained one, we can ask it to generate new text for us! Let's implement a helper function to generate new sentences:

In [218]:
def generate_sentence(model, senten_max_length):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    
    # Repeat until we get an end token and keep our sentences to less than senten_max_length words for now
    while (not new_sentence[-1] == word_to_index[sentence_end_token]) and len(new_sentence) < senten_max_length:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            #print(next_word_probs[-1][0])
            
            # correcting for abnormalities
            #abs_v = [-i if i <0 else i for i in next_word_probs[-1][0]] 
            #nrm_v = [i/sum(abs_v) for i in abs_v] 
            abs_v = [0 if i <0 else i for i in next_word_probs[-1][0]] 
            nrm_v = [i/sum(abs_v) for i in abs_v] 

            samples = np.random.multinomial(1, nrm_v)
            sampled_word = np.argmax(samples)
            
        new_sentence.append(sampled_word)

    #print(new_sentence)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    #print(sentence_str)
    return sentence_str

In [219]:
num_sentences = 10
senten_min_length = 7
senten_max_length = 20

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model, senten_max_length)
    print (" ".join(sent))

from other or at at more more people 's can to some no '' 're there * of
about there how there people * . * or there than : time people time n't only there
any at think 're than & people one
other than have to n't * have
at more from i or think ? from their their one more '' at have how no he
'' only about or '' also & an 're
`` can 's their : time * than i people people : more really have get 're from
he their : can to some their also any also at
's other know can think at any some no some have n't only other time to : know
time any any at , i or i it * . n't , get an have ,


Well, ok, we cannot program a chatGPT in an anaconda notebook, but that is essentially how chatGPT is programmed!

As you can see, no intelligence here. Just a database!

Our vanilla RNN  can't generate meaningful text because it has not been trained on all words.

After you train it on all words, you will find that it's unable to learn dependencies between words that are several steps apart. That's why RNNs failed to gain popularity when they were first invented. They were beautiful in theory but didn't work well in practice, and we didn't immediately understand why.

Fortunately, the difficulties in training RNNs are [much better understood](http://arxiv.org/abs/1211.5063) now. 

More sophisticated RNN models, such as LSTMs, are the current state of the art for many tasks. Today, in NLP, *Transformers* rule!

In [ ]:
import tarfile
  
# open file
file = tarfile.open('hin_mixed_2019_10K.tar.gz')
  
# extracting file
file.extractall('./data')
  
file.close()

# hin_mixed_2019_10K-sentences

In [220]:
import re
from nltk import tokenize

#alphabets= "([A-Za-z])"
#prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
#suffixes = "(Inc|Ltd|Jr|Sr|Co)"
#starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
#acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
#websites = "[.](com|net|org|io|gov|edu|me)"
#digits = "([0-9])"

#vocabulary_size = 8000
vocabulary_size = 3000

unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

def clean_roman_numerals(text):
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    return re.sub(pattern, '&', text)

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print( "Reading txt file...")
with open('data\hin_mixed_2019_10K\hin_mixed_2019_10K-sentences.txt', 'r', encoding="utf-8") as f:
    text = f.read()
    
    text = text.replace(",",".")
    text = text.replace(":",".")
    text = text.replace(";",".")
    text = text.replace("?",".")
    text = text.replace("!",".")
    
    text = clean_roman_numerals(text)
    
    #text = re.sub(prefixes,"\\1<prd>",text)
    #text = re.sub(websites,"<prd>\\1",text)
    #text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    #if "..." in text: text = text.replace("...","<prd><prd><prd>")
    #if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    #text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    #text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    #text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    #text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    #text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    #if "”" in text: text = text.replace(".”","”.")
    #if "\"" in text: text = text.replace(".\"","\".")
    #if "!" in text: text = text.replace("!\"","\"!")
    #if "?" in text: text = text.replace("?\"","\"?")
    #text = text.replace(".",".<stop>")
    #text = text.replace("?","?<stop>")
    #text = text.replace("!","!<stop>")
    #text = text.replace("<prd>",".")
    #sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    #sentences = [s.strip() for s in sentences]
    
    sentences = tokenize.sent_tokenize(text)
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    
print(  "Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print(  "Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print(  "\nExample sentence: '%s'" % sentences[0])
print(  "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

Reading txt file...
Parsed 6208 sentences.
Found 34479 unique words tokens.
Using vocabulary size 3000.
The least frequent word in our vocabulary is '66' and appeared 6 times.

Example sentence: 'SENTENCE_START 1	000 गज़ल (1) सोच ले तू किधर जा रहा है. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', '1', 'UNKNOWN_TOKEN', 'UNKNOWN_TOKEN', '(', '1', ')', 'सोच', 'ले', 'तू', 'UNKNOWN_TOKEN', 'जा', 'रहा', 'है', '.', 'SENTENCE_END']'


In [221]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\540999602.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\540999602.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])


In [222]:
x_example, y_example = X_train[1000], y_train[1000]
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print ("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START जिस में UNKNOWN_TOKEN को खत्म करने की ताकत हो .
[1, 343, 4, 2999, 6, 682, 28, 5, 1506, 25, 3]

y:
जिस में UNKNOWN_TOKEN को खत्म करने की ताकत हो . SENTENCE_END
[343, 4, 2999, 6, 682, 28, 5, 1506, 25, 3, 2]


In [223]:
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in X_train[10]]), X_train[10]))

x:
SENTENCE_START इस पर पूरे देश की UNKNOWN_TOKEN UNKNOWN_TOKEN हुई हैं .
[1, 20, 13, 380, 89, 5, 2999, 2999, 112, 18, 3]


In [224]:
np.random.seed(17)
model = RNN(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print (o.shape, o)

(11, 3000) [[0.00033152 0.00033647 0.00033349 ... 0.00033278 0.00033288 0.00033252]
 [0.00033625 0.00033148 0.00033669 ... 0.00032744 0.00033536 0.00033026]
 [0.00033022 0.00033356 0.0003333  ... 0.00033197 0.00033578 0.00032975]
 ...
 [0.00033787 0.00033199 0.00033517 ... 0.00033349 0.00033537 0.00033674]
 [0.00033306 0.00033255 0.00033407 ... 0.00033467 0.00033167 0.00033762]
 [0.00033662 0.00033116 0.00033269 ... 0.00033312 0.00033797 0.00033652]]


In [225]:
predictions = model.predict(X_train[10])
print (predictions.shape, predictions)
print ("x:\n%s" % (" ".join([index_to_word[x] for x in predictions])))

(11,) [ 605  899 2592 2812 1432 2900 1279 1022  554  702  865]
x:
सफल रंग ट्रस्ट परिस्थितियों बचाने समर्पण ममता कवि बेटी इनमें युवाओं


In [226]:
# Limit to 1000 examples to save time
print ("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print ("Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000]))

Expected Loss for random predictions: 8.006368


C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\1920175922.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  N = np.sum((len(y_i) for y_i in y))


Actual loss: 8.006620


In [227]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNN(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\2693848989.py:38: RuntimeWarning: invalid value encountered in double_scalars
  relative_error = np.abs(backprop_gradient - estimated_gradient) / (


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [228]:
np.random.seed(17)
model = RNN(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

43.1 ms ± 4.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [229]:
np.random.seed(17)

# Train on a small subset of the data to see what happens
model = RNN(vocabulary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\1920175922.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  N = np.sum((len(y_i) for y_i in y))


2023-04-02 17:52:35: Loss after num_examples_seen=0 epoch=0: 8.006711
2023-04-02 17:52:41: Loss after num_examples_seen=100 epoch=1: 6.406952
2023-04-02 17:52:46: Loss after num_examples_seen=200 epoch=2: 5.678251
2023-04-02 17:52:52: Loss after num_examples_seen=300 epoch=3: 5.474514
2023-04-02 17:52:58: Loss after num_examples_seen=400 epoch=4: 5.340279
2023-04-02 17:53:04: Loss after num_examples_seen=500 epoch=5: 5.230854
2023-04-02 17:53:10: Loss after num_examples_seen=600 epoch=6: 5.128836
2023-04-02 17:53:17: Loss after num_examples_seen=700 epoch=7: 5.053961
2023-04-02 17:53:22: Loss after num_examples_seen=800 epoch=8: 5.008823
2023-04-02 17:53:29: Loss after num_examples_seen=900 epoch=9: 4.974727


In [230]:
num_sentences = 10
senten_min_length = 7
senten_max_length = 20

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model, senten_max_length)
    print (" ".join(sent))

इसके तरह हो हैं इसके आज सरकार । पुलिस । । साल रहा समय आज ' इस लिए
कहा इसके पुलिस समय ’ दिया जब । साल इसके ' बाद बाद इसके लेकिन होने सरकार '
गया पुलिस लेकिन लिए जाने होने । सिंह ’ ’ समय था। हैं जब रूप हो होने रूप
सरकार ’ हैं उन्होंने लेकिन बाद इसके हैं। लिए आज समय कि लेकिन जाता के कहा लिए इसके
। रूप में से किसी का ’ कहा इसके रूप ' हैं। साल होने जब जब था। हो
। होने तरह सरकार वे आज पुलिस जब रूप इसके में लेकिन से होने सिंह । समय लेकिन
में अपने गया रूप सरकार हुए ' सरकार बाद किसी ’ तरह दिया कहा बाद तरह रूप '
। ' सरकार जब को कहा कि था। लेकिन अपने था। ‘ ' होने पुलिस रहा था। सरकार
गया ' इसके सरकार इसके ' वे होने । होने आज था। ’ तरह तरह दिया इसके ’
' दिया था। लिए लिए था। का कहा आप । ' इसके में जाने ’ के पुलिस से



# The Addition RNN using tf
Ok, now that we saw how things work inside of an RNN, let's use tensorflow instead of our own code and a simpler sequence rather than Shakespeare sonnets.

What could be simpler than adding two numbers? And it's a *sequence*!

<br />
<left>
<img src="ipynb.images/cash-register.png" width=200 />
</left>

From the [keras](https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py) people!

One of my favorite Genetic Algorithm (GA) implementations is learning addition and multiplication tables. Do you think an RNN can learn to ***add***? Is it really doing addition the way human children learn how to do it, or is it different? Please think about this, and let's discuss it. 

We are going to **train** an RNN to learn to add by giving its *tons of examples* of addition of numbers. After all, an addition is a **sequence** of two numbers (is that how children learn to add?). The third number is the **result**. From the first two numbers, we want to learn to generate the third. Let's see what happens.

For tf2, you *will* need to modify from:
```
from keras.models import Sequential
from keras import layers
```
to:
```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
```

My old tensorflow:

In [29]:
!pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\abhis\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 


My new tensorflow:

In [30]:
!pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\abhis\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [31]:
from __future__ import print_function
#from keras.models import Sequential
#from keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from six.moves import range

# One-Hot encodings

[One-hot](https://en.wikipedia.org/wiki/One-hot) encoding is the simplest way to encode information with `1`s and `0`s.

Vectors obtained through **one-hot encoding** are **binary**, **sparse** (mostly made of zeros) and very **high-dimensional** (same dimensionality as the number of tokens in the token vocabulary).

One-hot encoding words generally leads to vectors that are 20,000-dimensional or higher (capturing a vocabulary of 20,000 token in this case).

One-hot encoding is also known as [dummy variables](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)). We did a number of one-hot encodings in INFO 6105.

Consider the sentence `The cat sat on the mat`. The vocabulary (or unique words) in this sentence is (`cat`, `mat`, `on`, `sat`, `the`). To represent each word, we will create a zero vector with length equal to the vocabulary, then place a one in the index that corresponds to the word. This approach is shown in the following diagram.

<br />
<left>
<img src="ipynb.images/one-hot-example.png" width=400 />
</left>

To create a vector that contains the encoding of the sentence, we could then concatenate the one-hot vectors for each word.

>**Key point**: This approach is inefficient. A one-hot encoded vector is very **sparse** (meaning, most indices are zero). Imagine we have 10,000 words in the vocabulary. To one-hot encode each word, we would create a vector where 99.99% of the elements are zero.

In a second approach we might try to encode each word using **base 10** instead of **base 2**. Continuing the example above, we could assign 1 to `cat`, 2 to `mat`, and so on. We could then encode the sentence `The cat sat on the mat` as a **dense** vector like [5, 1, 4, 3, 5, 2]. This appoach is efficient. Instead of a sparse vector, we now have a dense one (where all elements are full). But it *is* simpler!

We'll take about more efficient encodings (embeddings)! For the time being, let's code a one-hot encoding class:

# A one-hot encoding python class

In [32]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

Example use-case:

In [33]:
set('abcdefghijklmnopqrstuvwxyz')

{'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [34]:
set(['the', 'cat', 'sat', 'on', 'the', 'mat'])

{'cat', 'mat', 'on', 'sat', 'the'}

Why do we use `set` representations in python?

Ok, let's encode!

In [35]:
words = 'abcdefghijklmnop'
wtable = CharacterTable(words)
wtable.encode('g', 1)

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [36]:
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [37]:
ctable.encode('123',3)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

# Generating training data

Now, let's generate 50,000 examples of addition of up-to-3 digits numbers. The maximum length of the result is:
```(python)
len(999 + 999) = len(1998) = 4
```

How do we generate a random 3-digit number? This way:

In [38]:
''.join(np.random.choice(list('0123456789')))

'5'

In [39]:
int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, 4))))

55

Ready?

In [40]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '34+78  '). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    
    # store
    questions.append(query)
    expected.append(ans)
    print(query, ans)
    
print('Total addition questions:', len(questions))

Generating data...
624+977 1601
2+795   797 
6+153   159 
84+85   169 
7+63    70  
62+92   154 
7+3     10  
557+22  579 
52+1    53  
32+75   107 
53+86   139 
67+38   105 
815+9   824 
4+921   925 
327+35  362 
656+8   664 
677+10  687 
132+4   136 
417+0   417 
2+475   477 
7+98    105 
780+915 1695
8+2     10  
2+4     6   
804+93  897 
59+52   111 
9+983   992 
751+56  807 
31+8    39  
3+44    47  
12+82   94  
0+32    32  
6+3     9   
73+6    79  
89+25   114 
0+4     4   
6+284   290 
39+937  976 
36+376  412 
5+84    89  
2+394   396 
8+14    22  
59+3    62  
97+745  842 
865+2   867 
2+7     9   
278+693 971 
8+660   668 
94+91   185 
9+0     9   
33+33   66  
47+4    51  
54+73   127 
96+115  211 
1+9     10  
8+4     12  
10+467  477 
9+21    30  
0+18    18  
961+7   968 
1+74    75  
5+8     13  
7+0     7   
73+24   97  
58+4    62  
69+50   119 
72+69   141 
15+11   26  
649+3   652 
0+61    61  
86+43   129 
8+35    43  
324+104 428 
9+94    103 
515+1   516 
967+5 

330+859 1189
521+0   521 
9+99    108 
999+6   1005
9+377   386 
49+465  514 
272+935 1207
58+46   104 
9+796   805 
9+517   526 
69+6    75  
5+51    56  
5+458   463 
93+36   129 
9+829   838 
93+80   173 
648+6   654 
460+98  558 
447+388 835 
7+32    39  
93+740  833 
9+63    72  
93+16   109 
10+63   73  
163+11  174 
6+279   285 
936+77  1013
132+346 478 
88+25   113 
206+33  239 
805+9   814 
312+734 1046
19+63   82  
212+35  247 
63+98   161 
1+507   508 
48+906  954 
3+418   421 
84+62   146 
889+524 1413
86+62   148 
9+331   340 
320+8   328 
891+37  928 
24+330  354 
84+4    88  
411+8   419 
18+543  561 
9+183   192 
799+37  836 
280+4   284 
986+638 1624
439+0   439 
71+824  895 
1+22    23  
28+18   46  
8+288   296 
367+874 1241
390+7   397 
2+19    21  
5+335   340 
960+749 1709
396+204 600 
327+51  378 
993+94  1087
30+24   54  
47+318  365 
30+810  840 
34+77   111 
693+8   701 
21+87   108 
18+231  249 
21+72   93  
25+4    29  
66+876  942 
475+59  534 
443+3   446 

8+964   972 
696+2   698 
194+0   194 
47+217  264 
727+7   734 
6+57    63  
66+59   125 
51+3    54  
27+97   124 
4+86    90  
94+472  566 
549+58  607 
11+854  865 
0+21    21  
413+8   421 
79+0    79  
995+801 1796
662+233 895 
78+862  940 
35+179  214 
37+34   71  
769+0   769 
97+93   190 
25+678  703 
564+4   568 
91+99   190 
434+4   438 
0+795   795 
405+20  425 
707+889 1596
594+8   602 
184+93  277 
3+49    52  
836+2   838 
116+52  168 
356+855 1211
836+1   837 
30+42   72  
987+2   989 
18+818  836 
676+1   677 
6+61    67  
6+433   439 
4+92    96  
678+523 1201
9+62    71  
8+354   362 
49+77   126 
740+0   740 
2+13    15  
5+114   119 
129+9   138 
661+71  732 
84+967  1051
84+86   170 
2+713   715 
9+895   904 
120+6   126 
273+79  352 
336+3   339 
66+1    67  
793+44  837 
68+35   103 
62+3    65  
278+50  328 
87+304  391 
628+9   637 
49+8    57  
613+3   616 
73+73   146 
302+93  395 
141+3   144 
57+722  779 
1+917   918 
596+28  624 
11+67   78  
44+946  990 

743+54  797 
926+4   930 
294+796 1090
507+9   516 
12+136  148 
188+275 463 
871+21  892 
88+5    93  
608+8   616 
471+49  520 
223+959 1182
58+76   134 
80+408  488 
87+887  974 
622+76  698 
67+18   85  
318+29  347 
451+4   455 
88+90   178 
26+922  948 
341+64  405 
483+48  531 
3+741   744 
60+46   106 
787+9   796 
731+84  815 
1+750   751 
46+12   58  
82+991  1073
89+67   156 
19+54   73  
277+262 539 
938+46  984 
4+991   995 
233+14  247 
6+95    101 
34+92   126 
963+24  987 
20+6    26  
815+48  863 
7+419   426 
61+15   76  
8+554   562 
0+218   218 
34+551  585 
11+94   105 
579+806 1385
496+2   498 
12+833  845 
793+763 1556
1+487   488 
132+128 260 
885+9   894 
286+141 427 
902+57  959 
967+914 1881
465+69  534 
815+19  834 
63+65   128 
54+147  201 
0+60    60  
6+679   685 
62+430  492 
28+50   78  
953+76  1029
49+65   114 
57+66   123 
15+79   94  
9+310   319 
696+435 1131
544+574 1118
9+651   660 
301+78  379 
9+25    34  
95+67   162 
749+775 1524
4+753   757 

907+95  1002
417+5   422 
93+225  318 
672+5   677 
54+808  862 
860+818 1678
6+757   763 
256+6   262 
17+63   80  
46+187  233 
575+1   576 
0+732   732 
845+7   852 
24+3    27  
1+844   845 
786+69  855 
3+870   873 
18+64   82  
34+88   122 
91+33   124 
33+1    34  
90+955  1045
421+9   430 
824+2   826 
257+38  295 
20+905  925 
75+38   113 
699+91  790 
537+182 719 
368+407 775 
156+2   158 
8+541   549 
2+949   951 
76+207  283 
530+647 1177
837+816 1653
0+941   941 
53+363  416 
173+45  218 
29+996  1025
11+473  484 
222+661 883 
423+135 558 
79+35   114 
85+820  905 
1+121   122 
271+155 426 
3+466   469 
41+27   68  
686+907 1593
58+198  256 
925+795 1720
40+69   109 
98+619  717 
452+1   453 
709+92  801 
510+70  580 
748+36  784 
336+2   338 
8+861   869 
44+32   76  
888+25  913 
75+19   94  
46+290  336 
3+596   599 
970+17  987 
41+52   93  
90+4    94  
802+24  826 
276+617 893 
4+616   620 
28+707  735 
463+2   465 
521+473 994 
912+8   920 
647+40  687 
4+78    82  

...

312+340 652 
643+284 927 
831+861 1692
37+590  627 
211+323 534 
572+735 1307
269+917 1186
47+839  886 
969+23  992 
201+24  225 
260+888 1148
31+210  241 
15+605  620 
42+298  340 
680+385 1065
7+677   684 
63+777  840 
205+547 752 
412+81  493 
774+33  807 
109+68  177 
877+535 1412
853+810 1663
531+585 1116
47+787  834 
272+8   280 
361+40  401 
85+433  518 
358+494 852 
932+65  997 
156+68  224 
77+442  519 
17+552  569 
18+683  701 
407+70  477 
229+92  321 
594+42  636 
652+382 1034
567+821 1388
92+213  305 
874+206 1080
61+911  972 
275+621 896 
583+79  662 
45+956  1001
81+214  295 
59+888  947 
781+631 1412
781+82  863 
631+593 1224
849+19  868 
510+173 683 
482+62  544 
605+280 885 
32+555  587 
589+615 1204
35+967  1002
796+94  890 
765+998 1763
144+227 371 
215+74  289 
484+40  524 
102+95  197 
920+686 1606
92+612  704 
466+11  477 
61+958  1019
239+77  316 
209+44  253 
29+83   112 
93+358  451 
148+616 764 
27+711  738 
217+519 736 
418+891 1309
23+564  587 
73+301  374 

950+80  1030
57+968  1025
76+537  613 
136+480 616 
656+551 1207
878+7   885 
906+460 1366
17+656  673 
373+74  447 
999+711 1710
83+296  379 
143+999 1142
567+57  624 
446+32  478 
31+146  177 
664+793 1457
289+404 693 
35+963  998 
54+186  240 
851+36  887 
746+41  787 
435+432 867 
346+69  415 
299+756 1055
423+210 633 
976+601 1577
271+59  330 
606+763 1369
402+855 1257
451+612 1063
699+913 1612
584+217 801 
474+36  510 
80+788  868 
145+64  209 
43+261  304 
726+847 1573
41+516  557 
79+494  573 
45+965  1010
142+75  217 
49+214  263 
58+949  1007
870+988 1858
968+773 1741
419+827 1246
31+50   81  
299+89  388 
22+611  633 
45+637  682 
50+368  418 
856+519 1375
907+96  1003
805+70  875 
918+26  944 
512+177 689 
474+687 1161
100+65  165 
878+87  965 
736+356 1092
96+748  844 
412+65  477 
258+59  317 
19+411  430 
147+44  191 
183+34  217 
62+352  414 
3+389   392 
76+139  215 
990+15  1005
57+749  806 
63+817  880 
760+20  780 
710+543 1253
92+584  676 
99+820  919 
848+36  884 

318+357 675 
623+6   629 
236+90  326 
504+271 775 
38+625  663 
16+227  243 
27+300  327 
16+875  891 
410+85  495 
15+314  329 
410+42  452 
92+294  386 
865+40  905 
83+337  420 
13+818  831 
284+855 1139
540+914 1454
4+178   182 
233+443 676 
784+26  810 
54+267  321 
156+543 699 
53+208  261 
734+236 970 
61+156  217 
669+801 1470
69+215  284 
86+809  895 
95+365  460 
316+581 897 
821+434 1255
787+855 1642
208+83  291 
4+390   394 
601+701 1302
373+72  445 
169+87  256 
367+46  413 
837+899 1736
411+22  433 
262+90  352 
251+89  340 
51+17   68  
66+328  394 
83+744  827 
15+310  325 
303+70  373 
820+188 1008
585+966 1551
619+559 1178
82+636  718 
349+541 890 
35+243  278 
205+42  247 
509+72  581 
566+855 1421
76+817  893 
264+41  305 
179+25  204 
79+337  416 
937+56  993 
924+11  935 
542+72  614 
21+667  688 
372+1   373 
574+328 902 
740+896 1636
109+606 715 
252+952 1204
597+321 918 
29+418  447 
27+571  598 
814+998 1812
33+928  961 
10+450  460 
649+296 945 
193+77  270 

370+19  389 
965+794 1759
977+537 1514
622+107 729 
708+681 1389
549+649 1198
76+527  603 
539+460 999 
7+378   385 
41+548  589 
875+827 1702
91+925  1016
540+157 697 
63+146  209 
519+480 999 
479+821 1300
78+168  246 
96+890  986 
186+850 1036
54+474  528 
445+450 895 
423+87  510 
52+63   115 
831+891 1722
387+926 1313
418+463 881 
91+894  985 
126+197 323 
53+882  935 
451+81  532 
735+630 1365
20+140  160 
9+132   141 
722+32  754 
465+80  545 
25+650  675 
391+369 760 
53+971  1024
378+59  437 
17+387  404 
54+525  579 
85+856  941 
264+71  335 
201+2   203 
44+294  338 
585+453 1038
464+325 789 
48+941  989 
18+398  416 
849+31  880 
260+529 789 
79+157  236 
511+792 1303
68+445  513 
50+315  365 
56+850  906 
92+270  362 
5+327   332 
428+263 691 
737+800 1537
33+894  927 
95+935  1030
962+35  997 
886+473 1359
11+445  456 
639+449 1088
716+737 1453
72+342  414 
5+133   138 
546+86  632 
248+244 492 
79+377  456 
83+655  738 
643+46  689 
85+853  938 
52+271  323 
482+803 1285

94+237  331 
673+76  749 
620+42  662 
36+849  885 
26+523  549 
941+59  1000
212+808 1020
40+344  384 
594+25  619 
100+75  175 
561+5   566 
91+424  515 
515+353 868 
115+771 886 
700+402 1102
23+773  796 
74+597  671 
7+287   294 
232+543 775 
123+133 256 
41+109  150 
433+82  515 
66+629  695 
286+771 1057
798+26  824 
845+638 1483
427+22  449 
64+292  356 
856+76  932 
50+709  759 
790+93  883 
660+99  759 
817+53  870 
91+521  612 
212+591 803 
13+744  757 
834+55  889 
980+91  1071
538+368 906 
91+319  410 
63+739  802 
85+88   173 
20+329  349 
43+758  801 
5+371   376 
699+53  752 
337+335 672 
252+36  288 
18+402  420 
938+20  958 
938+640 1578
583+46  629 
76+721  797 
64+135  199 
627+188 815 
72+899  971 
698+32  730 
833+73  906 
785+66  851 
80+688  768 
866+91  957 
89+227  316 
635+19  654 
886+940 1826
55+569  624 
159+3   162 
128+360 488 
49+952  1001
599+235 834 
76+338  414 
829+908 1737
45+567  612 
0+881   881 
68+562  630 
121+21  142 
912+15  927 
191+26  217 

Let's list our **input** sequence:

In [41]:
questions

['624+977',
 '2+795  ',
 '6+153  ',
 '84+85  ',
 '7+63   ',
 '62+92  ',
 '7+3    ',
 '557+22 ',
 '52+1   ',
 '32+75  ',
 '53+86  ',
 '67+38  ',
 '815+9  ',
 '4+921  ',
 '327+35 ',
 '656+8  ',
 '677+10 ',
 '132+4  ',
 '417+0  ',
 '2+475  ',
 '7+98   ',
 '780+915',
 '8+2    ',
 '2+4    ',
 '804+93 ',
 '59+52  ',
 '9+983  ',
 '751+56 ',
 '31+8   ',
 '3+44   ',
 '12+82  ',
 '0+32   ',
 '6+3    ',
 '73+6   ',
 '89+25  ',
 '0+4    ',
 '6+284  ',
 '39+937 ',
 '36+376 ',
 '5+84   ',
 '2+394  ',
 '8+14   ',
 '59+3   ',
 '97+745 ',
 '865+2  ',
 '2+7    ',
 '278+693',
 '8+660  ',
 '94+91  ',
 '9+0    ',
 '33+33  ',
 '47+4   ',
 '54+73  ',
 '96+115 ',
 '1+9    ',
 '8+4    ',
 '10+467 ',
 '9+21   ',
 '0+18   ',
 '961+7  ',
 '1+74   ',
 '5+8    ',
 '7+0    ',
 '73+24  ',
 '58+4   ',
 '69+50  ',
 '72+69  ',
 '15+11  ',
 '649+3  ',
 '0+61   ',
 '86+43  ',
 '8+35   ',
 '324+104',
 '9+94   ',
 '515+1  ',
 '967+5  ',
 '47+578 ',
 '151+4  ',
 '0+2    ',
 '2+437  ',
 '7+91   ',
 '56+39  ',
 '5+75   ',
 '2+

Let's list our **output** sequence:

In [42]:
expected

['1601',
 '797 ',
 '159 ',
 '169 ',
 '70  ',
 '154 ',
 '10  ',
 '579 ',
 '53  ',
 '107 ',
 '139 ',
 '105 ',
 '824 ',
 '925 ',
 '362 ',
 '664 ',
 '687 ',
 '136 ',
 '417 ',
 '477 ',
 '105 ',
 '1695',
 '10  ',
 '6   ',
 '897 ',
 '111 ',
 '992 ',
 '807 ',
 '39  ',
 '47  ',
 '94  ',
 '32  ',
 '9   ',
 '79  ',
 '114 ',
 '4   ',
 '290 ',
 '976 ',
 '412 ',
 '89  ',
 '396 ',
 '22  ',
 '62  ',
 '842 ',
 '867 ',
 '9   ',
 '971 ',
 '668 ',
 '185 ',
 '9   ',
 '66  ',
 '51  ',
 '127 ',
 '211 ',
 '10  ',
 '12  ',
 '477 ',
 '30  ',
 '18  ',
 '968 ',
 '75  ',
 '13  ',
 '7   ',
 '97  ',
 '62  ',
 '119 ',
 '141 ',
 '26  ',
 '652 ',
 '61  ',
 '129 ',
 '43  ',
 '428 ',
 '103 ',
 '516 ',
 '972 ',
 '625 ',
 '155 ',
 '2   ',
 '439 ',
 '98  ',
 '95  ',
 '80  ',
 '7   ',
 '941 ',
 '1022',
 '47  ',
 '25  ',
 '11  ',
 '977 ',
 '8   ',
 '131 ',
 '907 ',
 '48  ',
 '8   ',
 '16  ',
 '106 ',
 '601 ',
 '378 ',
 '15  ',
 '504 ',
 '340 ',
 '631 ',
 '295 ',
 '49  ',
 '78  ',
 '971 ',
 '70  ',
 '91  ',
 '81  ',
 '1004',
 

# Training and test data

Let's divide all our observations into a *training* set and a *test* set. We will now *encode* our numbers into **one-hot vectors**: 

The first dimension is the numbers of rows (observations).

The second dimension is the number of characters in each sequence.

The third dimension is the size of our alphabet of symbols. Here, it's `len('0123456789+ ')`. In our statistical sentence translation notebook, it's the size of the word dictionary (word-to-index mapping).

Let's allocate space for the input to the encoder `x`, and the input to the decoder, `y`. They're both 3D **tensors**: The first dimension is the row, and the other two dimensions are the thought-vector representation of the question or the answer.

Here's the first question in math format, and in thought-vector format:

In [43]:
questions[0]

'624+977'

In [44]:
ctable.encode(questions[0], MAXLEN)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

And here's the answer, in math format, and in thought-vector format:

In [45]:
expected[0]

'1601'

In [46]:
ctable.encode(expected[0], DIGITS + 1)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

Let's create the thought-vectors for our entire training dataset:

In [47]:
print('Vectorization into thought:')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)
print()

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)
print()

print('Example:')
print('The first row of input data is encoded internally as:')
print(x_train[0])
print()
print('The first row of output data is encoded internally as:')
print(y_train[0])
print()
print('These internal representations represent these signals:')
print(ctable.decode(x_train[0]))
print(ctable.decode(y_train[0]))

Vectorization into thought:


C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\3342980164.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
C:\Users\abhis\AppData\Local\Temp\ipykernel_27256\3342980164.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(questions), DIGITS + 1, len(chars)), dty

Training Data:
(45000, 7, 12)
(45000, 4, 12)

Validation Data:
(5000, 7, 12)
(5000, 4, 12)

Example:
The first row of input data is encoded internally as:
[[False False False False False  True False False False False False False]
 [False False False False False False False False  True False False False]
 [False False False False  True False False False False False False False]
 [False  True False False False False False False False False False False]
 [False False False False False  True False False False False False False]
 [False False False False False False False False False False False  True]
 [ True False False False False False False False False False False False]]

The first row of output data is encoded internally as:
[[False False False False False False  True False False False False False]
 [False False  True False False False False False False False False False]
 [False False False  True False False False False False False False False]
 [ True False False False False False 

# RNN Model

We now build our RNN (an LSTM, actually), using the `Keras` library. Just a single layer RNN!

The input to every LSTM layer must be three-dimensional:

The three dimensions of this input are:

- **Samples** (observations): One sequence is one sample. A **batch** is comprised of one or more samples.
- **Time Steps**: One time step is one point of observation in the sequence.
- **Features**: One feature is one observation at a time step.

Most often, time steps and features are the same, but we still need 3 dimensions because RNNs are programmed that way.

### For example:
The model below defines an input layer that expects 1 or more samples, 50 time steps, and 2 features, and encodes information into 32 neurons.
```(python)
model = Sequential()
model.add(LSTM(32, input_shape=(50, 2)))
```

### For example:
This could be a sequence of 10 values:
```(python)
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0
```
We can define this sequence of numbers as a NumPy array.
```(python)
from numpy import array
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
```
We can then use the `reshape()` function on the NumPy array to reshape this one-dimensional array into a three-dimensional array with 1 sample, 10 time steps, and 1 feature at each time step.

The reshape() function, when called on an array, takes one argument which is a tuple defining the new shape of the array. We cannot pass in any tuple of numbers; the reshape must evenly reorganize the data in the array.
```(python)
data = data.reshape((1, 10, 1))
```

Consider the case where you have multiple parallel series as input for your model.

### For example,
This could be two parallel series of 10 values:
```(python)
series 1: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0
series 2: 1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1
```
We can define these data as a matrix of 2 columns with 10 rows:
```(python)
from numpy import array
data = array([
	[0.1, 1.0],
	[0.2, 0.9],
	[0.3, 0.8],
	[0.4, 0.7],
	[0.5, 0.6],
	[0.6, 0.5],
	[0.7, 0.4],
	[0.8, 0.3],
	[0.9, 0.2],
	[1.0, 0.1]])
```
This data can be framed as 1 sample with 10 time steps and 2 features per timestep.

It can be reshaped as a 3D array as follows:
```(python)
data = data.reshape(1, 10, 2)
```

So let's code our RNN where each row ***input*** is `MAXLEN` timesteps and `len(chars)` features (our dictionary size), and each row ***output*** is `DIGITS + 1` timesteps and `len(chars)` features.

The input if easy enough to code:
```
RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars)))
```

To connect with the output, this is bit harder to grok:
```
model.add(layers.RepeatVector(DIGITS + 1))
```

That is because the decoder is programmed to accept a **3D tensor**!

In [48]:
# Try replacing with GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()

# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
# We require DIGITS + 1 output vectors for our result. We will use the same fully 
# connected layer (Dense) to output each vector. To use the same layer DIGITS + 1
# times, we wrap it in a TimeDistributed() wrapper layer
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 4, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 4, 128)            131584    
                                                                 
 time_distributed (TimeDistr  (None, 4, 12)            1548      
 ibuted)                                                         
                                                                 
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


We had to connect the encoder to the decoder and they *did not originally fit*!

With
```(python)
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
```

..the encoder takes in a **3D tensor** of 50,000 rows, MAXLEN timesteps, `len(char)` features, and...

..for each observation, the encoder will produce a **2D matrix** of 128 rows and `len(chars)` columns (after MAXLEN timesteps), while the decoder needs a **3D tensor**, not just 128 rows and `len(chars)` features, which it will then run over `DIGITS + 1` timesteps. That's a problem!

Keras' [RepeatVector](https://keras.io/layers/core/#repeatvector) layer is used like an adapter to fit the encoder and decoder. We configure `RepeatVector` to repeat the input `DIGITS + 1` times. This creates a 3D output comprised of `DIGITS + 1` copies of 128 x `len(char)` features, that we decode `DIGITS + 1` times using the same fully connected layer for each of the `DIGITS + 1` desired output vectors.

If you comment out the `RepeatVector` line, you will get:
```(python)
ValueError: Input 0 is incompatible with layer lstm_xx: expected ndim=3, found ndim=2
```
Try it out!

Finally, the fully connected output layer will use a softmax activation function to output values in the range \[0,1\], so some closer to `False`, and others closer to` True`. We will approximate by **rounding off** to 0 or 1 for our final prediction. 

>**Note**: Why not leverage one output for *each* time step for the input sequence, rather than one output for the *final* timestep of the input sequence (which is what we do here)? The alternate option would give us intermediate timestep information, which we don't leveerage here. This is discussed [here](https://github.com/fchollet/keras/issues/395).

# Note
If you're using version 2.1 or higher of tensorflow, you have to modify this tf1 api:
```
preds = model.predict_classes(rowx, verbose=0)
```
to:
```
preds = np.argmax(model.predict(rowx), axis=-1)
```

# Training

Let's train our RNN for 35 epochs.

In [49]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 2):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    
    # Select 10 samples from the validation set at random so we can visualize
    # errors with green and red boxes
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        #preds = model.predict_classes(rowx, verbose=0)
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        print("debugging:")
        print(type(preds[0]))
        print("now decoding...:")
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
1/1 [==============================] - 1s 1s/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 4+98    T 102  ☒ 10  
1/1 [==============================] - 0s 51ms/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 496+556 T 1052 ☒ 101 
1/1 [==============================] - 0s 47ms/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 3+2     T 5    ☒ 33  
1/1 [==============================] - 0s 45ms/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 142+59  T 201  ☒ 111 
1/1 [==============================] - 0s 46ms/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 457+87  T 544  ☒ 101 
1/1 [==============================] - 0s 48ms/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 0+996   T 996  ☒ 105 
1/1 [==============================] - 0s 48ms/step
debugging:
<class 'numpy.ndarray'>
now decoding...:
Q 16+632  T 648  ☒ 111 
1/1 [==============================] - 0s 

In [51]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 35):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    
    # Select 10 samples from the validation set at random so we can visualize
    # errors with green and red boxes
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        print("debugging:")
        print(type(preds[0]))
        print("now decoding...:")
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
352/352 [==============================] - 14s 38ms/step - loss: 1.6022 - accuracy: 0.4037 - val_loss: 1.5204 - val_accuracy: 0.4326
debugging:
<class 'numpy.ndarray'>
now decoding...:


TypeError: unhashable type: 'numpy.ndarray'

The loss is pretty low after only 30 epochs (and the samples are all correct), so I stopped the training!

Let's evaluate the accuracy of our model:

In [153]:
# evaluate the keras model
_, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %.2f' % (accuracy*100))

5000/5000 [==============================] - 7s 1ms/step
Accuracy: 97.59


99%.. Not bad.. We now know how to add numbers!

Let's add two numbers and test our network. We need to make sure we encode our numbers as vectors of the same type and shape used for training!

Let's pick two numbers at random, sum them up, and pad them to 7 characters long:

In [154]:
x = np.random.randint(0, 100)
y = np.random.randint(0, 100)
z = x + y
print(x,y,z)
print()

x_plus_y_buffered = str(x) + '+' + str(y) + (7 - len(str(x) + '+' + str(y))) * ' '
x_plus_y_buffered

53 20 73



'53+20  '

Let's do the same thing with the result:

In [155]:
z_buffered = str(z) + (4 - len(str(z))) * ' '
z_buffered

'73  '

Now let's encode these numbers in the internal representation of the neural net.

>**Note**: Every kind of sensory inpur, whether it's from your eyes, your ears, your nose,... get encoded in *internal* brain representations. So, we're doing exactly the same thing here. Except that the internal representations of sensory inputs in our brains happen at a darwinian scale. That goes to show that finding the *right* encoding may make a big difference in the training and have an effect on overall accuracy.

In [156]:
x_plus_y_encoded = ctable.encode(x_plus_y_buffered, 7)
x_plus_y_encoded

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [157]:
ctable.decode(x_plus_y_encoded)

'53+20  '

In [158]:
z_encoded = ctable.encode(z_buffered, 4)
z_encoded

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [159]:
ctable.decode(z_encoded)

'73  '

We need to convert to booleans, since that is how we trained our RNN:

In [160]:
x_plus_y = str(x) + '+' + str(y)
x_plus_y_tf = np.zeros((len(x_plus_y_buffered), len(chars)), dtype=np.bool)
x_plus_y_tf

array([[False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False]])

In [161]:
list(enumerate(x_plus_y_tf))

[(0, array([False, False, False, False, False, False, False, False, False,
         False, False, False])),
 (1, array([False, False, False, False, False, False, False, False, False,
         False, False, False])),
 (2, array([False, False, False, False, False, False, False, False, False,
         False, False, False])),
 (3, array([False, False, False, False, False, False, False, False, False,
         False, False, False])),
 (4, array([False, False, False, False, False, False, False, False, False,
         False, False, False])),
 (5, array([False, False, False, False, False, False, False, False, False,
         False, False, False])),
 (6, array([False, False, False, False, False, False, False, False, False,
         False, False, False]))]

In [162]:
ctable.encode('2', 1)

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [163]:
for i, sentence in enumerate(x_plus_y_tf):
    x_plus_y_tf[i] = ctable.encode(x_plus_y_buffered[i], 1)
x_plus_y_tf

array([[False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [164]:
z_tf = np.zeros((len(z_buffered), len(chars)), dtype=np.bool)
z_tf

array([[False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False]])

In [165]:
for i, sentence in enumerate(z_tf):
    z_tf[i] = ctable.encode(z_buffered[i], 1)
z_tf

array([[False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

We're now ready for a **forward step** through our RNN. `Keras`' `predict_classes()` API takes in an *array* of inputs, so we arbitrarily double up our input: 

In [166]:
x_plus_y_tf2 = np.array((x_plus_y_tf, x_plus_y_tf), dtype=np.bool)
x_plus_y_tf2

array([[[False, False, False, False, False, False, False,  True, False,
         False, False, False],
        [False, False, False, False, False,  True, False, False, False,
         False, False, False],
        [False,  True, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False,  True, False, False, False, False,
         False, False, False],
        [False, False,  True, False, False, False, False, False, False,
         False, False, False],
        [ True, False, False, False, False, False, False, False, False,
         False, False, False],
        [ True, False, False, False, False, False, False, False, False,
         False, False, False]],

       [[False, False, False, False, False, False, False,  True, False,
         False, False, False],
        [False, False, False, False, False,  True, False, False, False,
         False, False, False],
        [False,  True, False, False, False, False, False, False, False,

We're now ready to call `predict_classes()`:

In [167]:
preds = model.predict_classes(x_plus_y_tf2, verbose=0)
preds

array([[9, 5, 0, 0],
       [9, 5, 0, 0]], dtype=int64)

Let's decode that representation:

In [168]:
ctable.decode(preds[0], calc_argmax=False)

'73  '

That's the correct result! Our RNN *knows* how to add two numbers.

<br />
<left>
<img src="ipynb.images/surprise.gif" width=200 />
</left>

Did it learn addition the same way you did as a child? Think about this.

# CPU or GPU

Let's run some tests.

With the following `.theanorc` in `~home` on the Mac, and `.theanorc.txt` in `C:\Users\<username>` on windows, you compute on the CPU:
```(python)
[global]
device = cpu
```

You can verify with the following code:

In [ ]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

With the following though, can you compute on the GPU?
```(python)
[global]
floatX = float32
device = cuda

[gpuarray]
preallocate = 1
```

In [ ]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

# Comparing RNNs with *Dense* Models

Just to see how *different* RNNs are from densely connected models, let's jog our memory with a classic example:

A Dense (fully connected) Model would work this way:

A famous example dataset is that of the *prima indians*. The independent features $X$ are:

- Number of times pregnant
- Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- Diastolic blood pressure (mm Hg)
- Triceps skin fold thickness (mm)
- 2-Hour serum insulin (mu U/ml)
- Body mass index (weight in kg/(height in m)^2)
- Diabetes pedigree function
- Age (years)
- Output Variables (y):

The label (0 or 1) is a diagnostic, binary-valued variable indicating whether the patient shows signs of diabetes according to World Health Organization criteria.

We want to train a model to map rows of input variables $X$ to the output variable $y$:  $y = f(X)$.

Let's load the dataset:

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

dataset = loadtxt('data/pima-indians-diabetes.data.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

Let's define the keras model:

In [ ]:
model = Sequential()
# line below adds the first Dense layer and does 2 things: defines the input or visible layer, and the first hidden layer
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

We compile the model:

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

We train (or *fit*) our model on our loaded data:

In [ ]:
model.fit(X, y, epochs=150, batch_size=10)

we evaluate the performance of our dense network on the same dataset:

In [ ]:
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Let's make probability predictions with the model:

In [ ]:
predictions = model.predict(X)
# round predictions 
rounded = [str(round(x[0])) for x in predictions]
', '.join(rounded)

We can also make *class* (or *crisp*) predictions with the model:

In [ ]:
class_predictions = model.predict_classes(X)
exact = [str(round(x[0])) for x in class_predictions]
', '.join(exact)

# Homework
- Train a *dense* network to add two numbers

- Finish training the Shakespear RNN and produce a nice sounding sonnet. We'll read each others' sonnets in class.

- Start working on your final project: An RNN/CNN to predict LES snow.

- What's the difference between **dense** networks and **recurrent** networks, in your words? Please be able to answer this question!